In [1]:
# -*- coding: utf-8 -*-
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression  #Regressão Linear Simples
from sklearn.model_selection import train_test_split


#Função que recebe os Ys calculados pela regressão, que são os Percents, e os Ys reais que são os PL (perdas/lucros) de cada aposta
def SomaLog(Percent,PL):
    MIN_BANCA,MAX_BANCA= 0.01, 0.1
    return sum((MAX_BANCA if p>MAX_BANCA else p)*pl for p,pl in zip(Percent,PL) if p>=MIN_BANCA )



#Le o csv e filtra as colunas que serão utilizadas
df=pd.read_csv('../input/jogos_.csv')
df=df['s_g,s_c,s_da,s_s,s_r,d_g,d_c,d_da,d_s,goal,goal_diff,oddsU,probU,probU_diff,mod0,mod25,mod50,mod75,pl_u'.split(',')]

#Remove as linhas estão NaN, (preguiça de procurar, mas como é só um exemplo de uso tudo bem)
df.dropna(inplace=True)

#Mostra as 5 primeiras linhas
df.head(5)

,s_g,s_c,s_da,s_s,s_r,d_g,d_c,d_da,d_s,goal,goal_diff,oddsU,probU,probU_diff,mod0,mod25,mod50,mod75,pl_u
0,0.0,6.0,54.0,8.0,0.0,0.0,4.0,12.0,6.0,1.25,1.25,1.925,0.493421,0.006579,0.0,1.0,0.0,0.0,0.4625
1,0.0,2.0,66.0,6.0,0.0,0.0,2.0,10.0,4.0,1.25,1.25,1.950,0.486842,0.013158,0.0,1.0,0.0,0.0,-1.0000
2,1.0,8.0,34.0,9.0,0.0,1.0,8.0,0.0,3.0,2.50,1.50,1.750,0.539474,0.039474,0.0,0.0,1.0,0.0,-1.0000
3,3.0,9.0,52.0,6.0,0.0,1.0,5.0,2.0,0.0,4.25,1.25,1.975,0.480263,0.019737,0.0,1.0,0.0,0.0,-1.0000
4,2.0,4.0,28.0,11.0,0.0,0.0,0.0,6.0,5.0,3.50,1.50,1.900,0.500000,0.000000,0.0,0.0,1.0,0.0,-1.0000


Vamos dividir df entre conjunto de Treinamento e Teste de forma randômica, mas fixando o tamanho entre 50 mil registros. Será escolhido um estado randômico de forma que a média do pl_u do conjunto de teste esteja bem próxima da média do conjunto de treinamento.

In [2]:
#Divide entre treinamento e teste, foi escolhi o random_state=881, pois nesse as médias de Y_train e  Y_test estão bem próximas
df_train, df_test = train_test_split(df, test_size=50000, random_state=881)

print('pl_medio_train:', df_train.pl_u.mean(), '\npl_medio_test: ', df_test.pl_u.mean())

pl_medio_train: -0.02396081235105205 
pl_medio_test:  -0.023993450000000093


Vamos fazer uma Regressão Linear sem nenhum crtiério de seleção


In [3]:
X_train, Y_train=df_train.loc[:,df_train.columns!='pl_u'], df_train.pl_u
X_test, Y_test  =df_test.loc[:,df_test.columns!='pl_u'], df_test.pl_u

#Treinamos a regressão Linear
reg1=LinearRegression().fit(X_train,Y_train)


#Exibe a Lucratividade através do SomaLog
print('Lucratividade Todos os Jogos')
print( 'SomaLog: ', SomaLog( reg1.predict(X_test), Y_test)  )
print( 'SomaLog/mil jogos: ', SomaLog( reg1.predict(X_test), Y_test)/(len(Y_test)/1000  ) )



Lucratividade Todos os Jogos
SomaLog:  25.3739214558526
SomaLog/mil jogos:  0.507478429117052


Vamos agora fazer uma regressão linear filtrando apenas os jogos que estejam empatados ao intervalo, ou seja, dg=0

In [4]:
#Filtra apenas os jogos empatados
df_train=df_train[df_train.d_g==0]
df_test=df_test[df_test.d_g==0]

X_train, Y_train=df_train.loc[:,df_train.columns!='pl_u'], df_train.pl_u
X_test, Y_test  =df_test.loc[:,df_test.columns!='pl_u'], df_test.pl_u

#Treinamos a regressão Linear
reg1=LinearRegression().fit(X_train,Y_train)


#Exibe a Lucratividade através do SomaLog
print('Lucratividade Todos os Jogos')
print( 'SomaLog: ', SomaLog( reg1.predict(X_test), Y_test)  )
print( 'SomaLog/mil jogos: ', SomaLog( reg1.predict(X_test), Y_test)/(len(Y_test)/1000  ) )

Lucratividade Todos os Jogos
SomaLog:  20.14135583805771
SomaLog/mil jogos:  0.9849073759441423


Perceba que usar apenas os empates deixa nosso modelo muito mais lucrativo. Quase dobrando a relação SomaLog/mil jogos